In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("..")

import os
import numpy as np
import pandas as pd
import itertools
from multiprocessing import Pool

from src.utils import metrics, load_decompositions

datasets = sorted(["WikiText", "GitHub", "OpenWebText"])
topics = sorted([d + "_topics" for d in datasets])
models = sorted(["BERT", "GPT2", "BLOOM", "Llama2"])
index = pd.MultiIndex.from_product([models, datasets])

proj_dir = "out/"
topics_dir = "out-topics/"

dataset_models_comb = itertools.product(datasets, models)

In [ ]:
# RUN ONLY ONCE
def save_metrics(dataset, model):
    dataset, model = dataset.lower(), model.lower()
    out_dir = os.path.join(proj_dir, f"{dataset}-{model}")
    pos, cvec, global_mean = load_decompositions.load_pos_cvec_global_mean(
        dataset, model, proj_dir
    )
    results = metrics.ranks_and_explained_ratios_and_relative_norm(
        pos, cvec, global_mean
    )

    np.save(os.path.join(out_dir, "metrics.npy"), results)
    print(f"Finishing {dataset}+{model}")


with Pool() as p:
    p.starmap(save_metrics, dataset_models_comb)

In [ ]:
# RUN ONLY ONCE
topics_models_comb = list(itertools.product(topics, models))


def save_metrics(topic, model):
    topic, model = topic.lower(), model.lower()
    data_dir = os.path.join(topics_dir, f"{topic}-{model}")
    pos = np.load(f"{data_dir}/pos_id1-128_{model}.npy")
    L, T, C = pos.shape
    context_basis = np.memmap(
        f"{data_dir}/cbasis_id1-128_{model}.npy",
        mode="r",
        dtype=np.float32,
        shape=(L, 64 * 4, C),
    )

    B = metrics.avg_similarity_between_NoPCA(context_basis)
    W = metrics.avg_similarity_within_NoPCA(context_basis)

    np.save(f"{data_dir}/between_similarity_NoPCA.npy", B)
    np.save(f"{data_dir}/within_similarity_NoPCA.npy", W)
    print(f"Finishing {topic}+{model}")


with Pool() as p:
    p.starmap(save_metrics, topics_models_comb)

### Table 1 (screeNOT)

In [3]:
d = []
for model in models:
    for dataset in datasets:
        dataset, model = dataset.lower(), model.lower()
        out_dir = os.path.join(proj_dir, f"{dataset}-{model}")
        results = np.load(os.path.join(out_dir, "metrics.npy"))
        c = results[:, 0]
        if len(c) > 13:
            c = c[np.linspace(0, len(c) - 1, num=13, dtype=int)]
        d.append(c)

columns = [f"Layer {i}" for i in range(13)]
pd.DataFrame(np.array(d), index=index, columns=columns).astype(int)

Layer 0  Layer 1  Layer 2  Layer 3  Layer 4  Layer 5  \
BERT   GitHub            15       16       16       16       14       11   
       OpenWebText       15       16       18       16       11       11   
       WikiText          15       16       18       16       12       11   
BLOOM  GitHub             8        9        9        8        9       10   
       OpenWebText        6       10       10       11       11       10   
       WikiText           6        8       10       10        9       11   
GPT2   GitHub            15       14       13       12       12       11   
       OpenWebText       15       13       14       12       13       11   
       WikiText          15       14       14       13       11       11   
Llama2 GitHub             6       10        9        8       10        8   
       OpenWebText        7       10       10       11       11       10   
       WikiText           9       10       10       11       11        9   

                    Layer 6  Layer 7  Layer 8  Layer 9  Layer 10  Layer 11  \
BERT   GitHub            11        9       10       10        11        11   
       OpenWebText        9        9       11       11        11        11   
       WikiText           9        9       11       11        11        12   
BLOOM  GitHub            10       11       10       10        10        10   
       OpenWebText       11       11       11       11        10        10   
       WikiText          11       11       10       11        10        11   
GPT2   GitHub            11       10       10       10        11        11   
       OpenWebText       10       10       10       10         9         9   
       WikiText          11       11       11       10        10        10   
Llama2 GitHub             8        9        9        9         9         8   
       OpenWebText        9       10        9        9         9         8   
       WikiText           9        9        8        8         8         8   

                    Layer 12  
BERT   GitHub             12  
       OpenWebText        13  
       WikiText           12  
BLOOM  GitHub             10  
       OpenWebText        11  
       WikiText           11  
GPT2   GitHub             10  
       OpenWebText        12  
       WikiText           11  
Llama2 GitHub             10  
       OpenWebText        10  
       WikiText            9

### Table 1 (stable rank)

In [4]:
d = []
for model in models:
    for dataset in datasets:
        dataset, model = dataset.lower(), model.lower()
        out_dir = os.path.join(proj_dir, f"{dataset}-{model}")
        results = np.load(os.path.join(out_dir, "metrics.npy"))
        c = results[:, 1]
        if len(c) > 13:
            c = c[np.linspace(0, len(c) - 1, num=13, dtype=int)]
        d.append(c)

columns = [f"Layer {i}" for i in range(13)]
pd.DataFrame(np.array(d), index=index, columns=columns).round(2)

Layer 0  Layer 1  Layer 2  Layer 3  Layer 4  Layer 5  \
BERT   GitHub          9.19     7.79     5.26     4.73     4.34     3.84   
       OpenWebText     9.19     7.63     5.25     4.73     4.10     3.53   
       WikiText        9.19     7.77     5.05     4.60     4.01     3.49   
BLOOM  GitHub          8.39     1.25     1.20     1.21     1.21     1.23   
       OpenWebText     8.33     1.27     1.30     1.24     1.24     1.27   
       WikiText        7.75     1.27     1.28     1.28     1.28     1.32   
GPT2   GitHub          2.05     1.92     1.91     1.89     1.90     1.90   
       OpenWebText     2.05     1.92     1.91     1.89     1.88     1.88   
       WikiText        2.05     1.92     1.91     1.89     1.88     1.88   
Llama2 GitHub         24.87     1.00     1.00     1.00     1.00     1.00   
       OpenWebText    52.23     1.00     1.00     1.00     1.00     1.00   
       WikiText       26.26     1.00     1.00     1.01     1.01     1.02   

                    Layer 6  Layer 7  Layer 8  Layer 9  Layer 10  Layer 11  \
BERT   GitHub          3.48     3.20     2.70     2.45      2.04      1.84   
       OpenWebText     3.16     2.84     2.46     2.30      2.18      2.22   
       WikiText        3.15     2.83     2.43     2.28      2.14      2.17   
BLOOM  GitHub          1.29     1.29     1.28     1.25      1.21      1.02   
       OpenWebText     1.32     1.34     1.33     1.26      1.16      1.01   
       WikiText        1.39     1.40     1.39     1.31      1.20      1.01   
GPT2   GitHub          1.92     1.94     1.98     2.03      2.05      1.70   
       OpenWebText     1.88     1.90     1.91     1.96      2.02      2.24   
       WikiText        1.88     1.90     1.91     1.97      2.03      2.20   
Llama2 GitHub          1.00     1.01     1.01     1.01      1.02      1.03   
       OpenWebText     1.01     1.01     1.02     1.02      1.03      1.05   
       WikiText        1.04     1.06     1.11     1.18      1.22      1.27   

                    Layer 12  
BERT   GitHub           1.91  
       OpenWebText      2.15  
       WikiText         2.13  
BLOOM  GitHub           1.00  
       OpenWebText      1.00  
       WikiText         1.00  
GPT2   GitHub           1.11  
       OpenWebText      1.49  
       WikiText         1.57  
Llama2 GitHub           1.17  
       OpenWebText      1.44  
       WikiText         1.30

### Table 1 (Relative norm)

In [5]:
indices = pd.MultiIndex.from_product([models, datasets])

d = []
for model in models:
    for dataset in datasets:
        dataset, model = dataset.lower(), model.lower()
        out_dir = os.path.join(proj_dir, f"{dataset}-{model}")
        results = np.load(os.path.join(out_dir, "metrics.npy"))
        c = results[:, 3]
        if len(c) > 13:
            c = c[np.linspace(0, len(c) - 1, num=13, dtype=int)]
        d.append(c)

columns = [f"Layer {i}" for i in range(13)]
pd.DataFrame(np.array(d), index=indices, columns=columns).round(3)

Layer 0  Layer 1  Layer 2  Layer 3  Layer 4  Layer 5  \
BERT   GitHub         0.445    0.483    0.569    0.616    0.648    0.707   
       OpenWebText    0.465    0.546    0.660    0.759    0.877    0.977   
       WikiText       0.454    0.502    0.626    0.695    0.798    0.916   
BLOOM  GitHub         0.013    0.123    0.232    0.279    0.343    0.385   
       OpenWebText    0.012    0.138    0.194    0.264    0.315    0.342   
       WikiText       0.013    0.149    0.222    0.287    0.328    0.352   
GPT2   GitHub         0.999    0.994    0.996    0.972    0.812    0.762   
       OpenWebText    1.000    0.996    0.990    0.989    0.986    0.983   
       WikiText       1.000    0.995    0.991    0.989    0.987    0.986   
Llama2 GitHub         0.029    0.221    0.221    0.222    0.222    0.222   
       OpenWebText    0.038    0.146    0.146    0.146    0.147    0.147   
       WikiText       0.034    0.060    0.060    0.060    0.060    0.061   

                    Layer 6  Layer 7  Layer 8  Layer 9  Layer 10  Layer 11  \
BERT   GitHub         0.764    0.786    0.768    0.686     0.631     0.562   
       OpenWebText    0.973    0.967    0.953    0.901     0.777     0.658   
       WikiText       0.968    0.965    0.949    0.887     0.756     0.682   
BLOOM  GitHub         0.343    0.306    0.301    0.306     0.325     0.219   
       OpenWebText    0.297    0.267    0.264    0.300     0.392     0.575   
       WikiText       0.336    0.301    0.295    0.325     0.407     0.494   
GPT2   GitHub         0.672    0.600    0.489    0.442     0.386     0.303   
       OpenWebText    0.981    0.979    0.974    0.841     0.631     0.480   
       WikiText       0.984    0.984    0.981    0.933     0.680     0.555   
Llama2 GitHub         0.223    0.223    0.224    0.225     0.226     0.227   
       OpenWebText    0.148    0.148    0.150    0.152     0.154     0.154   
       WikiText       0.062    0.063    0.065    0.068     0.072     0.076   

                    Layer 12  
BERT   GitHub          0.473  
       OpenWebText     0.596  
       WikiText        0.627  
BLOOM  GitHub          0.181  
       OpenWebText     0.589  
       WikiText        0.491  
GPT2   GitHub          0.123  
       OpenWebText     0.075  
       WikiText        0.110  
Llama2 GitHub          0.183  
       OpenWebText     0.148  
       WikiText        0.191

### Table 1 (Inter & intra similarity)

In [7]:
row = []
for model in models:
    for topic in topics:
        topic, model = topic.lower(), model.lower()
        data_dir = os.path.join(topics_dir, f"{topic}-{model}")
        B = np.load(f"{data_dir}/between_similarity_NoPCA.npy")
        W = np.load(f"{data_dir}/within_similarity_NoPCA.npy")
        row.append([np.mean(B), np.std(B), np.mean(W), np.std(W)])

pd.DataFrame(
    np.array(row), index=index, columns=["B mean", "B std", "W mean", "W std"]
).round(2)

B mean  B std  W mean  W std
BERT   GitHub         0.17   0.04    0.31   0.05
       OpenWebText    0.13   0.04    0.26   0.04
       WikiText       0.12   0.04    0.28   0.06
BLOOM  GitHub         0.20   0.14    0.40   0.08
       OpenWebText    0.15   0.14    0.32   0.09
       WikiText       0.14   0.13    0.31   0.09
GPT2   GitHub         0.28   0.06    0.42   0.04
       OpenWebText    0.10   0.01    0.44   0.04
       WikiText       0.10   0.01    0.41   0.04
Llama2 GitHub         0.17   0.10    0.40   0.07
       OpenWebText    0.17   0.19    0.43   0.12
       WikiText       0.47   0.35    0.60   0.25